In [1]:
import numpy as np
from stable_koopman_operator import StableKoopmanOperator
from quad import Quad
from task import Task, Adjoint

import matplotlib.pyplot as plt
import scipy.linalg
from group_theory import VecTose3, TransToRp, RpToTrans
from lqr import FiniteHorizonLQR
from quatmath import euler2mat 

Helper Functions
================

In [2]:
def get_measurement(x):
    g = x[0:16].reshape((4,4)) ## SE(3) matrix
    R,p = TransToRp(g)
    twist = x[16:]
    grot = np.dot(R, [0., 0., -9.81]) ## gravity vec rel to body frame
    return np.concatenate((grot, twist)) 

def get_position(x):
    g = x[0:16].reshape((4,4))
    R,p = TransToRp(g)
    return p


In [3]:
quad = Quad() ### instantiate a quadcopter

### the timing parameters for the quad are used
### to construct the koopman operator and the adjoint dif-eq 
koopman_operator = StableKoopmanOperator(quad.time_step)
adjoint = Adjoint(quad.time_step)


In [4]:
np.abs(np.linalg.eig(koopman_operator.K)[0])

array([4.75309096, 4.75309096, 4.55730027, 4.55730027, 4.03203562,
       4.03203562, 3.85201156, 3.85201156, 3.70980676, 3.70980676,
       2.9904613 , 2.73359905, 2.73359905, 2.11312238, 2.11312238,
       1.81046212, 1.81046212, 0.94964777, 0.65360576, 2.72725555,
       2.72725555, 2.9591927 ])

In [5]:
_R = euler2mat(np.random.uniform(-1.,1., size=(3,)))
_p = np.array([0., 0., 0.])
_g = RpToTrans(_R, _p).ravel()
_twist = np.random.uniform(-1., 1., size=(6,)) #* 2.0
state = np.r_[_g, _twist]

In [11]:
for t in range(10):
    m_state = get_measurement(state)
    ustar = np.random.normal(0., 0.2, size=(4,))
    next_state = quad.step(state, ustar)
    koopman_operator.compute_operator_from_data(m_state, ustar, 
                                               get_measurement(next_state))
    state = next_state

13.381697184637583
13.438468771340219
13.50869844689601
13.413891255170826
13.554954736193709
13.445266432591895
13.576066260399028
13.502365040796008
13.560847490942662
13.56907845459143


In [29]:
np.abs(np.linalg.eig(koopman_operator.K)[0])

array([0.71607101, 0.69151388, 0.69151388, 0.7592198 , 0.7592198 ,
       0.83515118, 0.83515118, 0.55633573, 0.55633573, 0.62626633,
       0.62626633, 0.33627431, 0.33627431, 0.47808718, 0.47808718,
       0.54841893, 0.54841893, 0.40184543, 0.40184543, 0.62491019,
       0.50788494, 0.38131563])

In [25]:
np.linalg.eig(koopman_operator.Kx)[0]

array([-198.86728158+562.50864538j, -198.86728158-562.50864538j,
        -48.47830689+516.2126206j ,  -48.47830689-516.2126206j ,
       -344.6558852 +359.11143808j, -344.6558852 -359.11143808j,
         37.88240145+375.69320547j,   37.88240145-375.69320547j,
        -69.45688375+287.56937246j,  -69.45688375-287.56937246j,
       -126.35822926+222.87275818j, -126.35822926-222.87275818j,
         99.7211587   +0.j        , -274.80624452  +0.j        ,
       -218.39796443  +0.j        , -116.56067536  +0.j        ,
        -70.35636184  +0.j        ,  -78.69807891  +0.j        ])